In [6]:
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam


In [7]:
from tensorflow.keras.applications import VGG16

base = VGG16(
    weights="vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5",
    include_top=False,
    input_shape=(224,224,3)
)


In [8]:

# ---- Freeze base model ----
for layer in base.layers:
    layer.trainable = False


In [12]:
train_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

selected_classes = ['airplanes', 'ant']

train_data = train_gen.flow_from_directory(
    "caltech-101-img", target_size=(224,224),
    batch_size=32, subset='training', classes=selected_classes
)

val_data = train_gen.flow_from_directory(
    "caltech-101-img", target_size=(224,224),
    batch_size=32, subset='validation', classes=selected_classes
)


Found 674 images belonging to 2 classes.
Found 168 images belonging to 2 classes.


In [16]:

# ---- Add Classifier on top of base model ----
model = Sequential([
    base,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(train_data.num_classes, activation='softmax')
])

# ---- Compile (train only new layers) ----
model.compile(
    optimizer=Adam(learning_rate = 0.1),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [17]:

# ---- Train ----
model.fit(train_data, validation_data=val_data, epochs=5)


Epoch 1/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 93s 4s/step - accuracy: 0.8124 - loss: 3620.5239 - val_accuracy: 0.9524 - val_loss: 0.1482
Epoch 2/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 96s 4s/step - accuracy: 0.9133 - loss: 78.0893 - val_accuracy: 0.9524 - val_loss: 0.1483
Epoch 3/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 97s 4s/step - accuracy: 0.9397 - loss: 3.1008 - val_accuracy: 0.9524 - val_loss: 0.1575
Epoch 4/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 92s 4s/step - accuracy: 0.9550 - loss: 12.8416 - val_accuracy: 0.9524 - val_loss: 0.2301
Epoch 5/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 93s 4s/step - accuracy: 0.9425 - loss: 0.7520 - val_accuracy: 0.9524 - val_loss: 0.1917


In [18]:
loss, accuracy = model.evaluate(val_data)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 0.9469 - loss: 0.2126
Validation Loss: 0.1917
Validation Accuracy: 0.9524
